In [1]:
!pip install --no-cache-dir transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm

In [3]:
casa=pd.read_csv(r"/content/drive/MyDrive/Hack Corruption - Contractor/datos/limpio/value.csv",skiprows=1,header=None)

In [4]:
casa

,0,1
0,CONTRATACION DE SERVICIOS DE MANTENIMIENTO PRE...,271.268706
1,ADQUISICION DE JERINGAS DESECHABLES DE DIFEREN...,1709.999034
2,ADQUISICIÓN DE REACTIVOS DE INMUNOHISTOQUIMICA...,968.541223
3,ADQUISICION DE SET DE BOMBAS DE INFUSION VOLUM...,3830.404646
4,CONTRATACIÓN DE SERVICIO DE MANTENIMIENTO Y RE...,642.747446
...,...,...
21817,CONSTRUCCION DE BLOQUES DE SANITARIOS EN LA ES...,26.999841
21818,CONSTRUCCION DE LA SEDE DE LA DIRECCION DEPART...,217.962550
21819,Adquisición de acondicionadores de aire,5.811174
21820,Techado para cancha y construcción de escenari...,88.013407


In [5]:
pre=True

In [6]:
class BertDataset(Dataset):
    def __init__(self, tokenizer,max_length):
        super(BertDataset, self).__init__()
        self.train_csv=pd.read_csv(r"/content/drive/MyDrive/Hack Corruption - Contractor/datos/limpio/value.csv",skiprows=1,header=None)
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,0]

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'target': torch.tensor(self.train_csv.iloc[index, 1], dtype=torch.long)
            }
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-multilingual-cased")

dataset= BertDataset(tokenizer, max_length=100)
batch_size=64
dataloader=DataLoader(dataset=dataset,batch_size=batch_size)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [7]:
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.bert_model = transformers.BertModel.from_pretrained("bert-base-multilingual-cased")
        self.medium2 = nn.Linear(768, 3000)
        self.medium3 = nn.ReLU()
        self.medium4=nn.Linear(3000,768,bias=False)
        self.medium5 = nn.Sigmoid()
        self.out = nn.Linear(768, 1,bias=False)

    def forward(self,ids,mask,token_type_ids):
        _,o2= self.bert_model(ids,attention_mask=mask,token_type_ids=token_type_ids, return_dict=False)
        medium2=self.medium2(o2)
        medium3=self.medium3(medium2)
        medium4=self.medium4(medium3)
        out= self.out(medium4)

        return out

model=BERT()

loss_fn = nn.L1Loss()

#Initialize Optimizer
optimizer= optim.Adam(model.parameters(),lr= 1e-8)


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [8]:
for param in model.bert_model.parameters():
    param.requires_grad = False

In [9]:
def finetune(epochs,dataloader,model,loss_fn,optimizer):
  model.train()
  for  epoch in range(epochs):
    print(epoch)

    loop=tqdm(enumerate(dataloader),leave=False,total=len(dataloader))
    for batch, dl in loop:
      ids=dl['ids']
      token_type_ids=dl['token_type_ids']
      mask= dl['mask']
      label=dl['target']
      label = label.unsqueeze(1)

      optimizer.zero_grad()

      output=model(
          ids=ids,
          mask=mask,
          token_type_ids=token_type_ids)
      label = label.type_as(output)

      loss=loss_fn(output,label)
      loss.backward()

      optimizer.step()

      pred = output

      num_correct = sum((a-b)**2 for a, b in zip(pred, label) )/batch_size
      num_samples = pred.shape[0]
      accuracy = num_correct/(1e-6*batch_size)

      print(output[:4])
      print(label[:4])

      print(f'Got {num_correct} ')
      # Show progress while training
      loop.set_description(f'Epoch={epoch}/{epochs}')
      loop.set_postfix(loss=loss.item(),acc=accuracy)
    if True:
      torch.save(model.state_dict(), "/content/drive/MyDrive/Hack Corruption - Contractor/modelos/primero.pt")
  return model

In [10]:
try:
  model.load_state_dict(torch.load("/content/drive/MyDrive/Hack Corruption - Contractor/modelos/primero.pt"))
except Exception as e:
  print(e)


In [ ]:
model=finetune(10, dataloader, model, loss_fn, optimizer)


0


  0%|          | 0/341 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Epoch=0/10:   0%|          | 1/341 [00:38<3:40:33, 38.92s/it

tensor([[18.0243],
        [24.8491],
        [26.1175],
        [25.1670]], grad_fn=<SliceBackward0>)
tensor([[ 271.],
        [1709.],
        [ 968.],
        [3830.]])
Got tensor([1407613.2500], grad_fn=<DivBackward0>) 


Epoch=0/10:   1%|          | 2/341 [01:04<2:55:11, 31.01s/it, acc=tensor([9.8814e+11], grad_fn=<DivBackward0>), loss=1.52e+3]

tensor([[28.3047],
        [27.2507],
        [31.3307],
        [24.6474]], grad_fn=<SliceBackward0>)
tensor([[12.],
        [14.],
        [90.],
        [56.]])
Got tensor([63241236.], grad_fn=<DivBackward0>) 


...

tensor([[27.2679],
        [22.9934],
        [25.5404],
        [23.3463]], grad_fn=<SliceBackward0>)
tensor([[81.],
        [ 7.],
        [24.],
        [15.]])
Got tensor([47555.3008], grad_fn=<DivBackward0>) 


Epoch=5/10:  16%|█▌        | 53/341 [21:32<1:56:26, 24.26s/it, acc=tensor([3.0848e+11], grad_fn=<DivBackward0>), loss=570]

tensor([[28.1872],
        [25.6471],
        [26.6408],
        [28.5577]], grad_fn=<SliceBackward0>)
tensor([[ 4.],
        [ 6.],
        [28.],
        [27.]])
Got tensor([19742888.], grad_fn=<DivBackward0>) 


Epoch=5/10:  16%|█▌        | 54/341 [21:56<1:55:12, 24.09s/it, acc=tensor([3.2480e+12], grad_fn=<DivBackward0>), loss=2.35e+3]

tensor([[27.8297],
        [28.1342],
        [26.5216],
        [26.1105]], grad_fn=<SliceBackward0>)
tensor([[3.5581e+04],
        [2.5000e+01],
        [2.7000e+01],
        [8.0000e+00]])
Got tensor([2.0787e+08], grad_fn=<DivBackward0>) 


In [ ]:
# TODO https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [ ]:
help(model)

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
model.eval()

In [ ]:
if True:
  torch.save(model.state_dict(), "/content/drive/MyDrive/Hack Corruption - Contractor/modelos/primero.pt")

In [ ]:
"""
0
  0%|          | 0/341 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Epoch=0/10:   0%|          | 1/341 [00:29<2:47:30, 29.56s/it, acc=tensor([2.2047e+10], grad_fn=<DivBackward0>), loss=383]tensor([[20.2252],
        [22.6409],
        [24.1670],
        [27.0135]], grad_fn=<SliceBackward0>)
tensor([[ 271.],
        [1709.],
        [ 968.],
        [3830.]])
Got tensor([1411026.3750], grad_fn=<DivBackward0>)
Epoch=0/10:   1%|          | 2/341 [00:51<2:20:44, 24.91s/it, acc=tensor([9.8818e+11], grad_fn=<DivBackward0>), loss=1.52e+3]tensor([[31.9692],
        [29.4062],
        [19.1494],
        [20.2431]], grad_fn=<SliceBackward0>)
tensor([[12.],
        [14.],
        [90.],
        [56.]])
Got tensor([63243708.], grad_fn=<DivBackward0>)
Epoch=0/10:   1%|          | 3/341 [01:12<2:12:05, 23.45s/it, acc=tensor([9.1024e+09], grad_fn=<DivBackward0>), loss=219]tensor([[27.2450],
        [22.8395],
        [29.0450],
        [18.8878]], grad_fn=<SliceBackward0>)
tensor([[  25.],
        [  25.],
        [ 269.],
        [2275.]])
Got tensor([582555.], grad_fn=<DivBackward0>)
Epoch=0/10:   1%|          | 4/341 [01:33<2:06:17, 22.49s/it, acc=tensor([8.4780e+09], grad_fn=<DivBackward0>), loss=170]tensor([[24.4433],
        [19.0652],
        [21.5157],
        [33.4835]], grad_fn=<SliceBackward0>)
tensor([[19.],
        [53.],
        [55.],
        [ 9.]])
Got tensor([542591.7500], grad_fn=<DivBackward0>)
Epoch=0/10:   1%|▏         | 5/341 [01:56<2:05:44, 22.45s/it, acc=tensor([7.8610e+09], grad_fn=<DivBackward0>), loss=239]tensor([[28.8879],
        [20.4650],
        [24.3325],
        [30.4206]], grad_fn=<SliceBackward0>)
tensor([[  28.],
        [   9.],
        [2107.],
        [  73.]])
Got tensor([503106.1562], grad_fn=<DivBackward0>)
Epoch=0/10:   2%|▏         | 6/341 [02:18<2:05:03, 22.40s/it, acc=tensor([4.5726e+10], grad_fn=<DivBackward0>), loss=267]tensor([[24.3271],
        [27.2743],
        [27.6631],
        [24.5888]], grad_fn=<SliceBackward0>)
tensor([[  3.],
        [ 51.],
        [137.],
        [ 16.]])
Got tensor([2926485.5000], grad_fn=<DivBackward0>)
Epoch=0/10:   2%|▏         | 7/341 [02:39<2:02:27, 22.00s/it, acc=tensor([2.6902e+11], grad_fn=<DivBackward0>), loss=704]tensor([[19.3495],
        [15.5978],
        [30.8341],
        [21.5261]], grad_fn=<SliceBackward0>)
tensor([[0.0000e+00],
        [5.2300e+02],
        [3.2813e+04],
        [1.9000e+01]])
Got tensor([17217098.], grad_fn=<DivBackward0>)
Epoch=0/10:   2%|▏         | 8/341 [03:02<2:03:51, 22.32s/it, acc=tensor([3.3873e+10], grad_fn=<DivBackward0>), loss=353]tensor([[20.0164],
        [38.3046],
        [29.5814],
        [26.1379]], grad_fn=<SliceBackward0>)
tensor([[ 8.],
        [ 5.],
        [ 4.],
        [73.]])
Got tensor([2167872.], grad_fn=<DivBackward0>)
Epoch=0/10:   3%|▎         | 9/341 [03:24<2:02:30, 22.14s/it, acc=tensor([2.2864e+11], grad_fn=<DivBackward0>), loss=839]tensor([[25.0058],
        [31.9741],
        [27.3741],
        [30.4187]], grad_fn=<SliceBackward0>)
tensor([[   92.],
        [25494.],
        [   49.],
        [ 1157.]])
Got tensor([14633154.], grad_fn=<DivBackward0>)
Epoch=0/10:   3%|▎         | 10/341 [03:45<2:00:03, 21.76s/it, acc=tensor([2.1704e+09], grad_fn=<DivBackward0>), loss=99.5]tensor([[32.7043],
        [25.9091],
        [25.0380],
        [25.3637]], grad_fn=<SliceBackward0>)
tensor([[ 6.],
        [18.],
        [21.],
        [45.]])
Got tensor([138906.6562], grad_fn=<DivBackward0>)
Epoch=0/10:   3%|▎         | 11/341 [04:13<2:10:24, 23.71s/it, acc=tensor([9.9544e+10], grad_fn=<DivBackward0>), loss=462]tensor([[31.5790],
        [24.5337],
        [21.2282],
        [37.7488]], grad_fn=<SliceBackward0>)
tensor([[2.0041e+04],
        [1.3000e+01],
        [5.6800e+02],
        [9.4000e+01]])
Got tensor([6370832.], grad_fn=<DivBackward0>)
Epoch=0/10:   4%|▎         | 12/341 [04:33<2:04:06, 22.63s/it, acc=tensor([6.9570e+09], grad_fn=<DivBackward0>), loss=257]tensor([[28.3190],
        [31.0277],
        [35.3385],
        [17.0330]], grad_fn=<SliceBackward0>)
tensor([[ 65.],
        [ 10.],
        [498.],
        [ 49.]])
Got tensor([445249.5938], grad_fn=<DivBackward0>)
Epoch=0/10:   4%|▍         | 13/341 [04:54<2:01:19, 22.19s/it, acc=tensor([1.3280e+09], grad_fn=<DivBackward0>), loss=85.4]tensor([[24.6177],
        [31.3669],
        [22.8789],
        [34.9246]], grad_fn=<SliceBackward0>)
tensor([[13.],
        [24.],
        [ 4.],
        [ 0.]])
Got tensor([84993.9141], grad_fn=<DivBackward0>)
Epoch=0/10:   4%|▍         | 14/341 [05:18<2:03:18, 22.62s/it, acc=tensor([2.0449e+10], grad_fn=<DivBackward0>), loss=202]tensor([[21.5922],
        [31.1598],
        [25.8999],
        [31.4450]], grad_fn=<SliceBackward0>)
tensor([[ 10.],
        [  6.],
        [226.],
        [ 28.]])
Got tensor([1308740.1250], grad_fn=<DivBackward0>)
Epoch=0/10:   4%|▍         | 15/341 [05:40<2:01:24, 22.34s/it, acc=tensor([3.5432e+10], grad_fn=<DivBackward0>), loss=260]tensor([[34.0648],
        [25.3877],
        [28.8457],
        [22.6140]], grad_fn=<SliceBackward0>)
tensor([[ 32.],
        [  7.],
        [ 24.],
        [161.]])
Got tensor([2267668.], grad_fn=<DivBackward0>)
Epoch=0/10:   5%|▍         | 16/341 [06:01<1:58:50, 21.94s/it, acc=tensor([3.0699e+10], grad_fn=<DivBackward0>), loss=320]tensor([[30.5350],
        [21.1430],
        [23.6539],
        [27.3951]], grad_fn=<SliceBackward0>)
tensor([[29.],
        [49.],
        [28.],
        [ 4.]])
Got tensor([1964714.6250], grad_fn=<DivBackward0>)
Epoch=0/10:   5%|▍         | 17/341 [06:23<1:59:22, 22.11s/it, acc=tensor([34614524.], grad_fn=<DivBackward0>), loss=22.7]tensor([[26.8710],
        [24.7808],
        [31.0662],
        [30.0878]], grad_fn=<SliceBackward0>)
tensor([[ 6.],
        [11.],
        [26.],
        [ 5.]])
Got tensor([2215.3296], grad_fn=<DivBackward0>)
Epoch=0/10:   5%|▌         | 18/341 [06:46<1:59:43, 22.24s/it, acc=tensor([1.3489e+09], grad_fn=<DivBackward0>), loss=84.7]tensor([[26.9312],
        [26.3201],
        [23.6060],
        [22.6836]], grad_fn=<SliceBackward0>)
tensor([[ 3.],
        [22.],
        [ 4.],
        [23.]])
Got tensor([86331.8281], grad_fn=<DivBackward0>)
Epoch=0/10:   6%|▌         | 19/341 [07:07<1:57:49, 21.96s/it, acc=tensor([8.9578e+08], grad_fn=<DivBackward0>), loss=101]tensor([[28.0742],
        [25.3192],
        [21.0753],
        [23.4542]], grad_fn=<SliceBackward0>)
tensor([[1316.],
        [ 132.],
        [  63.],
        [   3.]])
Got tensor([57329.8242], grad_fn=<DivBackward0>)
Epoch=0/10:   6%|▌         | 20/341 [07:28<1:56:29, 21.78s/it, acc=tensor([2.0870e+10], grad_fn=<DivBackward0>), loss=436]tensor([[29.4567],
        [23.2891],
        [19.6259],
        [24.1375]], grad_fn=<SliceBackward0>)
tensor([[46.],
        [ 3.],
        [ 3.],
        [21.]])
Got tensor([1335694.8750], grad_fn=<DivBackward0>)
Epoch=0/10:   6%|▌         | 21/341 [07:52<1:58:54, 22.29s/it, acc=tensor([2.4161e+09], grad_fn=<DivBackward0>), loss=94.2]tensor([[27.4590],
        [29.1668],
        [17.9188],
        [15.5728]], grad_fn=<SliceBackward0>)
tensor([[  14.],
        [ 582.],
        [2893.],
        [  49.]])
Got tensor([154629.6719], grad_fn=<DivBackward0>)
Epoch=0/10:   6%|▋         | 22/341 [08:13<1:56:36, 21.93s/it, acc=tensor([8.3579e+09], grad_fn=<DivBackward0>), loss=181]tensor([[24.9096],
        [34.4704],
        [23.9591],
        [30.1057]], grad_fn=<SliceBackward0>)
tensor([[ 5.],
        [35.],
        [35.],
        [25.]])
Got tensor([534908.1875], grad_fn=<DivBackward0>)
Epoch=0/10:   7%|▋         | 23/341 [08:34<1:55:04, 21.71s/it, acc=tensor([9.6949e+08], grad_fn=<DivBackward0>), loss=116]tensor([[36.6938],
        [29.2972],
        [28.0808],
        [23.1473]], grad_fn=<SliceBackward0>)
tensor([[ 11.],
        [318.],
        [  3.],
        [  6.]])
Got tensor([62047.5898], grad_fn=<DivBackward0>)
Epoch=0/10:   7%|▋         | 24/341 [08:58<1:57:44, 22.29s/it, acc=tensor([1.4262e+09], grad_fn=<DivBackward0>), loss=67.6]tensor([[29.0340],
        [26.3242],
        [25.5459],
        [36.5160]], grad_fn=<SliceBackward0>)
tensor([[  89.],
        [  41.],
        [  50.],
        [2415.]])
Got tensor([91279.6641], grad_fn=<DivBackward0>)
Epoch=0/10:   7%|▋         | 25/341 [09:19<1:56:13, 22.07s/it, acc=tensor([2.3404e+11], grad_fn=<DivBackward0>), loss=707]tensor([[30.5117],
        [25.7422],
        [23.0695],
        [25.7255]], grad_fn=<SliceBackward0>)
tensor([[  0.],
        [  0.],
        [  4.],
        [784.]])
Got tensor([14978289.], grad_fn=<DivBackward0>)
Epoch=0/10:   8%|▊         | 26/341 [09:41<1:54:47, 21.87s/it, acc=tensor([4.3880e+08], grad_fn=<DivBackward0>), loss=65.3]tensor([[23.6833],
        [29.1710],
        [28.1763],
        [23.0232]], grad_fn=<SliceBackward0>)
tensor([[261.],
        [ 71.],
        [ 47.],
        [ 25.]])
Got tensor([28082.9004], grad_fn=<DivBackward0>)
Epoch=0/10:   8%|▊         | 27/341 [10:04<1:56:32, 22.27s/it, acc=tensor([7.7199e+09], grad_fn=<DivBackward0>), loss=246]tensor([[15.4225],
        [23.9016],
        [28.2712],
        [25.4006]], grad_fn=<SliceBackward0>)
tensor([[149.],
        [760.],
        [128.],
        [830.]])
Got tensor([494075.3125], grad_fn=<DivBackward0>)
Epoch=0/10:   8%|▊         | 28/341 [10:32<2:04:35, 23.88s/it, acc=tensor([9.7473e+08], grad_fn=<DivBackward0>), loss=95.4]tensor([[27.9405],
        [33.6821],
        [29.2644],
        [33.8423]], grad_fn=<SliceBackward0>)
tensor([[14.],
        [21.],
        [24.],
        [14.]])
Got tensor([62382.9609], grad_fn=<DivBackward0>)
Epoch=0/10:   9%|▊         | 29/341 [10:53<1:59:52, 23.05s/it, acc=tensor([7.7028e+09], grad_fn=<DivBackward0>), loss=117]tensor([[29.1998],
        [26.5003],
        [14.4626],
        [26.2526]], grad_fn=<SliceBackward0>)
tensor([[   8.],
        [ 101.],
        [  55.],
        [5620.]])
Got tensor([492980.0625], grad_fn=<DivBackward0>)
Epoch=0/10:   9%|▉         | 30/341 [11:16<1:59:51, 23.12s/it, acc=tensor([3.1716e+10], grad_fn=<DivBackward0>), loss=398]tensor([[27.8302],
        [28.1186],
        [22.4863],
        [26.7432]], grad_fn=<SliceBackward0>)
tensor([[  25.],
        [  25.],
        [3140.],
        [ 920.]])
Got tensor([2029815.8750], grad_fn=<DivBackward0>)
Epoch=0/10:   9%|▉         | 31/341 [11:38<1:57:53, 22.82s/it, acc=tensor([2.1375e+09], grad_fn=<DivBackward0>), loss=74.6]tensor([[22.3190],
        [25.0774],
        [34.3143],
        [25.2233]], grad_fn=<SliceBackward0>)
tensor([[15.],
        [ 3.],
        [86.],
        [10.]])
Got tensor([136798.0312], grad_fn=<DivBackward0>)
Epoch=0/10:   9%|▉         | 32/341 [12:00<1:55:14, 22.38s/it, acc=tensor([2.0518e+10], grad_fn=<DivBackward0>), loss=287]tensor([[26.9008],
        [30.4082],
        [31.2319],
        [22.3332]], grad_fn=<SliceBackward0>)
tensor([[17.],
        [ 9.],
        [74.],
        [22.]])
Got tensor([1313160.2500], grad_fn=<DivBackward0>)
Epoch=0/10:  10%|▉         | 33/341 [12:22<1:55:11, 22.44s/it, acc=tensor([2.5494e+10], grad_fn=<DivBackward0>), loss=430]tensor([[23.8404],
        [26.7578],
        [27.9991],
        [27.2060]], grad_fn=<SliceBackward0>)
tensor([[1054.],
        [ 100.],
        [ 190.],
        [6665.]])
Got tensor([1631643.8750], grad_fn=<DivBackward0>)
Epoch=0/10:  10%|▉         | 34/341 [12:45<1:55:14, 22.52s/it, acc=tensor([2.4274e+08], grad_fn=<DivBackward0>), loss=42.3]tensor([[29.4335],
        [30.9162],
        [24.2261],
        [26.7126]], grad_fn=<SliceBackward0>)
tensor([[939.],
        [ 67.],
        [328.],
        [207.]])
Got tensor([15535.1211], grad_fn=<DivBackward0>)
Epoch=0/10:  10%|█         | 35/341 [13:06<1:52:42, 22.10s/it, acc=tensor([5.6832e+08], grad_fn=<DivBackward0>), loss=62.2]tensor([[31.2970],
        [23.4740],
        [22.2165],
        [31.0224]], grad_fn=<SliceBackward0>)
tensor([[ 24.],
        [ 62.],
        [228.],
        [ 48.]])
Got tensor([36372.6406], grad_fn=<DivBackward0>)
Epoch=0/10:  11%|█         | 36/341 [13:28<1:52:28, 22.13s/it, acc=tensor([9.4339e+10], grad_fn=<DivBackward0>), loss=531]tensor([[23.0801],
        [23.5647],
        [26.3112],
        [31.4308]], grad_fn=<SliceBackward0>)
tensor([[1610.],
        [  75.],
        [ 829.],
        [ 268.]])
Got tensor([6037674.5000], grad_fn=<DivBackward0>)
"""